# Arrow and DuckDB Example Notebook

Demonstrating using Apache Arrow with the DuckDB client via SQL.

## References

- [Apache Arrow](https://arrow.apache.org/docs/index.html)
- [DuckDB](https://duckdb.org/docs/)
- [SQL](https://en.wikipedia.org/wiki/SQL)
- [Iris Dataset](https://archive.ics.uci.edu/ml/datasets/Iris)
    - Creator: R.A. Fisher
    - Donor: Michael Marshall (MARSHALL%PLU '@' io.arc.nasa.gov)

In [1]:
import pathlib
import urllib

import duckdb
import pyarrow.csv as csv
import pyarrow.parquet as parquet

data_link = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"

In [2]:
# write a parquet file from iris CSV data
parquet.write_table(
    table=csv.read_csv(
        input_file=urllib.request.urlopen(data_link),
        read_options=csv.ReadOptions(
            column_names=[
                "sepal length",
                "sepal width",
                "petal length",
                "petal width",
                "class",
            ]
        ),
    ),
    where="iris.parquet",
)

In [3]:
# show our resulting parquet file
list(pathlib.Path(".").glob("*.parquet"))

[PosixPath('iris.parquet')]

In [4]:
# use duckdb to select sepal data with row number as a pyarrow table
sepal_table = duckdb.connect().execute(
    f"""
    SELECT file_row_number, "sepal length", "sepal width"
    FROM read_parquet('iris.parquet', file_row_number=true);
    """
).arrow()
sepal_table

pyarrow.Table
file_row_number: int64
sepal length: double
sepal width: double
----
file_row_number: [[0,1,2,3,4,...,145,146,147,148,149]]
sepal length: [[5.1,4.9,4.7,4.6,5,...,6.7,6.3,6.5,6.2,5.9]]
sepal width: [[3.5,3,3.2,3.1,3.6,...,3,2.5,3,3.4,3]]

In [5]:
# use duckdb to select petal data with row number as a pyarrow table
petal_table = duckdb.connect().execute(
    f"""
    SELECT file_row_number, "petal length", "petal width"
    FROM read_parquet('iris.parquet', file_row_number=true);
    """
).arrow()
petal_table

pyarrow.Table
file_row_number: int64
petal length: double
petal width: double
----
file_row_number: [[0,1,2,3,4,...,145,146,147,148,149]]
petal length: [[1.4,1.4,1.3,1.5,1.4,...,5.2,5,5.2,5.4,5.1]]
petal width: [[0.2,0.2,0.2,0.2,0.2,...,2.3,1.9,2,2.3,1.8]]

In [6]:
# use duckdb to select iris class data with row number as a pyarrow table
class_table = duckdb.connect().execute(
    f"""
    SELECT file_row_number, class
    FROM read_parquet('iris.parquet', file_row_number=true);
    """
).arrow()
class_table

pyarrow.Table
file_row_number: int64
class: string
----
file_row_number: [[0,1,2,3,4,...,145,146,147,148,149]]
class: [["Iris-setosa","Iris-setosa","Iris-setosa","Iris-setosa","Iris-setosa",...,"Iris-virginica","Iris-virginica","Iris-virginica","Iris-virginica","Iris-virginica"]]

In [7]:
# use duckdb to join pyarrow table data using the python variable names as tables
joined_result = duckdb.connect().execute(
    f"""
    SELECT 
        sepal_table."sepal length",
        sepal_table."sepal width",
        petal_table."petal length",
        petal_table."petal width",
        class_table.class
    FROM sepal_table
    LEFT JOIN petal_table ON
        petal_table.file_row_number = sepal_table.file_row_number
    LEFT JOIN class_table ON
        class_table.file_row_number = sepal_table.file_row_number
    """
).arrow()
joined_result

pyarrow.Table
sepal length: double
sepal width: double
petal length: double
petal width: double
class: string
----
sepal length: [[5.1,4.9,4.7,4.6,5,...,6.7,6.3,6.5,6.2,5.9]]
sepal width: [[3.5,3,3.2,3.1,3.6,...,3,2.5,3,3.4,3]]
petal length: [[1.4,1.4,1.3,1.5,1.4,...,5.2,5,5.2,5.4,5.1]]
petal width: [[0.2,0.2,0.2,0.2,0.2,...,2.3,1.9,2,2.3,1.8]]
class: [["Iris-setosa","Iris-setosa","Iris-setosa","Iris-setosa","Iris-setosa",...,"Iris-virginica","Iris-virginica","Iris-virginica","Iris-virginica","Iris-virginica"]]

In [8]:
# double check that our joined result is the same as the original parquet file
joined_result.equals(parquet.read_table("iris.parquet"))

True